# Train Model


This is a generic notebook, that lets you train models of different classes.

You can also easily use this notebook to perform adversarial training, or to train a "smooth" classifier for randomized smoothing.

#### Preamble

In [5]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:


import numpy as np

import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers


import config
from utils.RobustnessExperiment import load_module
from utils.Visual import show_audio, show_spectogram, show_waveplot

from classification.trainer.HyperParamSearch import SaveCallback


#### Setup Dataset Handler

In [9]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

#### Define Save Callback
To always keep the best model

In [10]:
cb = SaveCallback("best_model")

### Train the Model

In [22]:
# MODELS
from classification.models.M5 import M5PLModule
from classification.models.SpectrogramCNN_8K import SpectrogramCNN_8KPLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.CRNN_8k import CRNN8kPLModule
from classification.models.CRNN import CRNNPLModule
from classification.models.DeepRecursiveCNN8k import DeepRecursiveCNN8kPLModule
from classification.models.DeepRecursiveCNN import DeepRecursiveCNNPLModule

# ATTACKS
from attacks.pgd import ProjectedGradientDescent, PGD_Real

print("Currently Best:", cb.best_val_acc)

############################################################
##### Specify Hyper Parameters
############################################################
# these are the best hyperparameters we found for the CRNN8k model.
# have a look at the wiki, to find the best hyperparameters for our other models!
hparams = {
 'batch_size': 64,
 'learning_rate': 0.00091159593618428,
 'lr_decay': 0.701004802632707,
 'weight_decay': 0.020050519680598525,
 'p_dropout': 0.18416455677220223,
 'lstm_hidden_size': 32,
 'epochs': 3 # just for demo - better use ~50
}


############################################################
##### Specify model class
############################################################
model = CRNN8kPLModule(hparams)
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')

############################################################
##### Extra Options
############################################################

########
# uncomment this line, to train a "smooth" classifier for randomized smoothing
# i.e., add gaussian noise with std=sigma during training
########
#model.set_smooth(sigma=0.5)

########
# adversarial training: uncomment here, to train with adversarial samples
########
#model.setAttack(PGD_Real, {"norm": 1, "epsilon": 1})
#model.setAttack(ProjectedGradientDescent, {"norm": "inf", "epsilon": {"SAMPLING_MODE": "choice", "options": [0.01, 0.1]}})

trainer = pl.Trainer(
    max_epochs=hparams["epochs"],
    logger= loggers.TensorBoardLogger(config.LOG_DIR, name=type(model).__name__),
    gpus=1 if torch.cuda.is_available() else None,
    callbacks=[cb]
)

trainer.fit(model)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name                      | Type                  | Params
----------------------------------------------------------------
0  | model                     | CRNN8k                | 258 K 
1  | model.spec                | MelspectrogramStretch | 0     
2  | model.spec.spectrogram    | Spectrogram           | 0     
3  | model.spec.mel_scale      | MelScale              | 0     
4  | model.spec.stft           | Spectrogram           | 0     
5  | model.spec.random_stretch | RandomTimeStretch     | 0     
6  | model.spec.complex_norm   | ComplexNorm           | 0     
7  | model.spec.norm           | SpecNormalization     | 0     
8  | model.convs               | Sequential            | 213 K 
9  | model.convs.0             | Conv2d                | 320   
10 | model.convs.1             | BatchNorm2d           | 64    
11 | model.convs.2       

Currently Best: 0.890337877889745


Val-Acc=0.02133965619442798


Val-Acc=0.8304682868998222
Train-Acc=0.7938735177865612


Val-Acc=0.8275044457617071
Train-Acc=0.8367588932806325


Val-Acc=0.8713692946058091
Train-Acc=0.8541501976284585



1

#### Saving a model

In [25]:
print(model)
print("parameters:", sum(p.numel() for p in model.parameters()))
model.save("my_model.p")

CRNN8kPLModule(
  (model): CRNN8k(
    (spec): MelspectrogramStretch()
    (convs): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ELU(alpha=1.0)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ELU(alpha=1.0)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout(p=0.18416455677220223, inplace=False)
      (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
      (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ELU(alpha=1.0)
      (11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (13): ELU(alpha=1.0)
      (14): MaxPool2d(kernel_size=2, stride=2,

### Load a model

In [35]:
#model = load_module("/nfs/homedirs/herrmanp/shortcut/SAVED_MODELS/CRNN8k/CRNN8k_vanilla_0.8951.p", CRNN8kPLModule)
model = load_module("/nfs/homedirs/herrmanp/shortcut/SAVED_MODELS/CRNN8k/CRNN8k_attack_L_inf_eps=0.03_iters=2_v11.p", CRNN8kPLModule)

datasetHandler.load(model, 'validation')    #, old_data = True)
datasetHandler.load(model, 'training')      #, old_data = True)

print(model.hparams)

model.cuda()
model.report()

{'batch_size': 64, 'learning_rate': 0.0009, 'weight_decay': 0.001, 'lr_decay': 0.95, 'epochs': 50, 'p_dropout': 0.36, 'n_hidden': 10, 'lstm_hidden_size': 64}
Accuracy: 	0.87
Precision: 	0.90
Recall: 	0.84
F1-Score: 	0.87

VAL-ACC: 1475/1687 (87.43331357439241%)

P-Rate: 	0.46


{'tp': array(693),
 'fp': array(79),
 'tn': array(782),
 'fn': array(133),
 'correct': array(1475),
 'n': 1687,
 'acc': 0.8743331357439241,
 'prec': 0.8976683937823834,
 'rec': 0.8389830508474576,
 'f1': 0.867334167709637,
 'attack_args': None,
 'p_rate': 0.4576170717249555}

# Run a single attack!

In [36]:
from attacks.volume import VolumeAttack
from attacks.pitch import PitchAttack
from attacks.speed import TimeStretchAttack
from attacks.pgd import ProjectedGradientDescent, PGD_Real
from torch.utils.data import DataLoader, random_split


val_dataloader =  DataLoader(model.val_dataloader().dataset, shuffle=True, batch_size=20, num_workers=1)
attack = ProjectedGradientDescent(model.model, val_dataloader, {"epsilon":0.1, "norm": "inf"}, early_stopping=-1, device='cuda', save_samples=True)

attack.attack()
attack.report( )

100%|██████████| 85/85 [00:02<00:00, 39.02it/s]

Attack-Parameters:	{'epsilon': 0.1, 'norm': 'inf'}
Early stopping: 	False (-1)

Successfully attacked:	110
Total attacked: 	1484
Total processed:	1687

Success-Rate: 		0.07
Perturbed Accurracy: 	0.81


{'success_rate': 0.07412398921832884, 'acc': 0.8144635447540012}